In [2]:
# Ý tưởng:
# Biểu diễn mỗi ký tự bằng one-hot
# Ở mỗi bước t:
#     Thực hiện tính công thức trạng thái ẩn
#     Đầu ra đưa qua softmax
#     loss là cross-entropy loss
# Backpropagation: Tính gradient từ T -> 1, cộng dồn qua thời gian
# Có gradient clipping tránh nổ gradient
import numpy as np

In [3]:
def one_hot(idx, vocab_size):
    # vector v có vocab_size hàng và 1 cột
    v = np.zeros((vocab_size, 1))
    # Cả hàng idx sẽ toàn giá trị 1.0
    v[idx] = 1.0
    return v
def softmax(z):
    exp = np.exp(z)
    return exp / np.sum(exp)


In [4]:
v = np.zeros((3, 2))
v[1] = 1.0
v

array([[0., 0.],
       [1., 1.],
       [0., 0.]])

In [ ]:
# RNN thuần
class RNN:
    def __init__(self, input_size, hidden_size, output_size):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        # Khởi tạo trọng số gồm value + và -
        s = 0.01
        # ma trận gồm hidden_size hàng, input_size cột
        self.Wxh = np.random.randn(hidden_size, input_size) * s
        # ma trận gồm hidden_size hàng, hidden_size cột
        self.Whh = np.random.randn(hidden_size, hidden_size) * s
        self.bh = np.zeros((hidden_size, 1))
        # ma trận gồm output_size hàng, hidden_size cột
        self.Why = np.random.randn(output_size, hidden_size) * s
        self.by = np.zeros((output_size, 1))
    def params(self):
        return {
            "Wxh": self.Wxh, "Whh": self.Whh, "bh": self.bh,
            "Why": self.Why, "by": self.by
        }
    def forward(self, inputs, prev_hidden_state):
        # inputs: list chỉ số của ký tự [x1, x2,...]
        # prev_hidden_state : hidden state ban đầu
        # return cache(xs, hs, os, ps) và hidden last
        # xs, hs, os, ps chứa các xt, ht, ot, pt trong công thức của RNN
        xs, hs, os, ps = {}, {}, {}, {}
        # hidden state trước t = 0
        hs[-1] = np.copy(prev_hidden_state)
        loss = 0.0
        for t, ix in enumerate(inputs):
            xs[t] = one_hot(ix, self.input_size)
            hs[t] = np.tanh(self.Wxh @ xs[t] + self.Whh @ hs[t-1] + self.bh)
            os[t] = self.Why @ hs[t] + self.by
            ps[t] = softmax(os[t])
        cache = (xs, hs, os, ps)
        # trả về hidden state cuối cùng làm input cho batch tiếp theo
        # trả về cache để tính backpropagation
        return cache, hs[len(inputs)-1]

In [26]:
wxh = np.random.randn(3, 5)
wxh

array([[ 0.80579132, -0.90663356,  0.37308253,  1.16266981, -0.89835884],
       [-0.810582  ,  0.32257693,  1.01016125,  2.06237602, -0.11625224],
       [ 0.39497624, -0.85410169,  0.82109994, -0.3229887 ,  0.46284877]])

In [14]:
s = 0.01
input_size = 4
hidden_size = 3
output_size = input_size
# ma trận gồm hidden_size hàng, input_size cột
Wxh = np.random.randn(hidden_size, input_size) * s
print("Wxh (hidden size, input size): \n",Wxh,"\n")
# ma trận gồm hidden_size hàng, hidden_size cột
Whh = np.random.randn(hidden_size, hidden_size) * s
print("Whh (hidden size, hidden size):\n",Whh, "\n")
bh = np.zeros((hidden_size, 1))
print("bh (hidden size):\n",bh, "\n")
# ma trận gồm output_size hàng, hidden_size cột
Why = np.random.randn(output_size, hidden_size) * s
print("Why (output size, hidden size): \n",Why, "\n")
by = np.zeros((output_size, 1))
print("by (output size): \n",by, "\n")

Wxh (hidden size, input size): 
 [[-0.00042016  0.00357459  0.03410074  0.00546874]
 [ 0.00519983 -0.01778254  0.02237733  0.00146076]
 [-0.00726405  0.00468883  0.00584563  0.00294953]] 

Whh (hidden size, hidden size):
 [[-0.00361037 -0.00688821  0.00387512]
 [ 0.00329453  0.00275857  0.00398298]
 [-0.00901703  0.01193357 -0.01026172]] 

bh (hidden size):
 [[0.]
 [0.]
 [0.]] 

Why (output size, hidden size): 
 [[-0.02105567 -0.02204825  0.00102543]
 [-0.00441272 -0.01103623 -0.01846537]
 [-0.02391485 -0.02176742  0.01963459]
 [ 0.01702913 -0.00356572 -0.01715876]] 

by (output size): 
 [[0.]
 [0.]
 [0.]
 [0.]] 



In [23]:
inputs = [1, 2, 3]
input_size = 4
xs, hs, os, ps = {}, {}, {}, {}
hs[-1] = np.zeros((hidden_size,1))
for t, ix in enumerate(inputs):
    xs[t] = one_hot(ix, input_size)
    hs[t] = np.tanh(Wxh @ xs[t] + Whh @ hs[t-1] + bh)
    print("giá trị thứ",t,"của input và hidden state thứ",t,"của input:\n",xs[t],"\n",hs[t])
    print("\n")
print(hs)

giá trị thứ 0 của input và hidden state thứ 0 của input:
 [[0.]
 [1.]
 [0.]
 [0.]] 
 [[ 0.00357458]
 [-0.01778066]
 [ 0.0046888 ]]


giá trị thứ 1 của input và hidden state thứ 1 của input:
 [[0.]
 [0.]
 [1.]
 [0.]] 
 [[0.03421512]
 [0.02235501]
 [0.00555303]]


giá trị thứ 2 của input và hidden state thứ 2 của input:
 [[0.]
 [0.]
 [0.]
 [1.]] 
 [[0.00521269]
 [0.00165727]
 [0.0028508 ]]


{-1: array([[0.],
       [0.],
       [0.]]), 0: array([[ 0.00357458],
       [-0.01778066],
       [ 0.0046888 ]]), 1: array([[0.03421512],
       [0.02235501],
       [0.00555303]]), 2: array([[0.00521269],
       [0.00165727],
       [0.0028508 ]])}
